In [22]:
from importlib import reload
from model import dataset

reload(dataset)

# Define the configuration for the Data Loader
LOADER_CONFIG = {
    'batch_size': 32,
    'use_data_cache': True
}

# Create the Data Loader
train_loader, test_loader, label_map = dataset.load(LOADER_CONFIG)
num_classes = sum(label is not None for label in label_map)

print(f"Num Train Samples: {len(train_loader)}")
print(f"Num Test Samples: {len(test_loader)}")
print(f"Num Labels: {sum(label is not None for label in label_map)}")

Num Train Samples: 1077
Num Test Samples: 264
Num Labels: 11


In [23]:
from importlib import reload
from model import bsnn

reload(bsnn)

BSNN_CONFIG = {
    'num_blocks': 2,
    'num_classes': num_classes,
    'img_dimension': 28,
}

model = bsnn.BSNN(config=BSNN_CONFIG)
print(model)